In [1]:
import cv2
import numpy as np

In [3]:
image = cv2.imread('images/receipt_image.jpg', 0)
color = cv2.imread('images/receipt_image.jpg')
hsv = cv2.cvtColor(color, cv2.COLOR_BGR2HSV)

hue, saturation, value = cv2.split(hsv)

In [2]:
def show(image):
    
    resized_height = 720
    percent = resized_height / len(image)
    resized_width = int(percent * len(image[0]))
    gray = cv2.resize(image,(resized_width,resized_height))


    cv2.imshow('cringe', gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [9]:
thresh = cv2.threshold(hue, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
show(thresh)

eroded = cv2.erode(thresh, kernel=None, iterations=7)
dilated = cv2.dilate(eroded, kernel=None, iterations=15)
eroded = cv2.erode(dilated, kernel=None, iterations=100)
dilated = cv2.dilate(eroded, kernel=None, iterations=100)

In [ ]:
show(dilated)

True

In [13]:
def extract_largest_rectangular_contour(color_image, preprocessed_image):
    contours, _ = cv2.findContours(preprocessed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = None
    largest_area = 0
    largest_approx = None
    
    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        if len(approx) == 4:
            area = cv2.contourArea(contour)
            if area > largest_area:
                largest_area = area
                largest_contour = contour
                largest_approx = approx
                
    if largest_contour is not None:
        cv2.drawContours(color_image, [largest_approx], -1, (0, 255, 0), 3)
        x, y, w, h = cv2.boundingRect(largest_contour)
        cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        return color_image, largest_approx
    
    return color_image, None

In [15]:
im, cor = extract_largest_rectangular_contour(color, dilated)
show(im)

In [20]:
show(image[370:1221, 306:708])

In [21]:
cropped = image[370:1221, 306:708]

In [30]:
show(cropped)

In [27]:
# thresh_cropped = cv2.threshold(cropped, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
thresh_cropped = cv2.adaptiveThreshold(cropped, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
show(thresh_cropped)

In [29]:
med_blur = cv2.medianBlur(thresh_cropped, 3)
show(med_blur)